**Mount Google Drive & Setup**

In [1]:
from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive')

# Define paths
REPO_PATH = "/content/PlanckDeep"
DATA_PATH = "/content/PlanckDeep/data"
DRIVE_DATA_DIR = "/content/drive/MyDrive/PlanckDeep/processed_data"

# Clone your repo (if not done already)
!git clone https://github.com/KhamessiTaha/PlanckDeep.git

# Create /data directory inside the repo
os.makedirs(DATA_PATH, exist_ok=True)

# Copy processed .npy files from Drive to /data
!cp {DRIVE_DATA_DIR}/*.npy {DATA_PATH}/


Mounted at /content/drive
Cloning into 'PlanckDeep'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 115 (delta 63), reused 89 (delta 37), pack-reused 0 (from 0)
Receiving objects: 100% (115/115), 3.12 MiB | 31.63 MiB/s, done.
Resolving deltas: 100% (63/63), done.


**Install dependencies**

In [2]:
!pip install numpy matplotlib scikit-learn torch torchvision tqdm seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

**Check data loaded properly**

In [3]:
import numpy as np
import os

patches = np.load(os.path.join(DATA_PATH, "cmb_patches_enhanced.npy"))
labels = np.load(os.path.join(DATA_PATH, "cmb_temperature_labels.npy"))

print("✓ Loaded patches:", patches.shape)
print("✓ Loaded labels:", labels.shape)
print("Unique labels:", np.unique(labels))
print("Class distribution:", np.bincount(labels))

✓ Loaded patches: (5000, 128, 128)
✓ Loaded labels: (5000,)
Unique labels: [0 1]
Class distribution: [3332 1668]


In [4]:
# Determine if binary classification
is_binary = len(np.unique(labels)) == 2
print(f"Binary classification: {is_binary}")

Binary classification: True


**Train models from script**

In [10]:
%cd /content/PlanckDeep/src

!python train_all_models.py \
  --patch_file ../data/cmb_patches_enhanced.npy \
  --label_file ../data/cmb_temperature_labels.npy \
  --models improved_cnn resnet physics_informed \
  --epochs 20 \
  --batch_size 32 \
  --use_class_weights \
  --use_early_stopping \
  --lr_scheduler cosine \
  --experiment_name deep_cmb_colab_run \
  --binary_classification


/content/PlanckDeep/src
Using device: cuda
GPU: Tesla T4
GPU Memory: 14.74 GB
Experiment directory: experiments/deep_cmb_colab_run
Binary classification mode: True
Loading and preparing datasets...
Loaded 5000 patches with shape (128, 128)
Number of classes: 2
Class distribution: [3332 1668]
Dataset loaded: 5000 patches
Patch shape: torch.Size([1, 128, 128])
Class distribution: [3332, 1668]
Dataset splits - Train: 3500, Val: 1000, Test: 500
Positive class weight: 2.0198

Training IMPROVED_CNN Model

Creating improved_cnn model...
Model parameters: 10,649,281 total, 10,649,281 trainable
Epoch 1/20 (6.51s) - Train Loss: 0.932390, Train Acc: 0.5923, Val Loss: 0.925628, Val Acc: 0.6719, LR: 1.00e-03
Epoch 2/20 (5.04s) - Train Loss: 0.929439, Train Acc: 0.6269, Val Loss: 0.924283, Val Acc: 0.4609, LR: 9.94e-04
Epoch 3/20 (4.42s) - Train Loss: 0.928361, Train Acc: 0.4753, Val Loss: 0.926063, Val Acc: 0.5596, LR: 9.76e-04
Epoch 4/20 (4.98s) - Train Loss: 0.927706, Train Acc: 0.4915, Val Loss:

**Save experiment results back to Drive**

In [11]:
# Copy the full experiment directory back to Drive
EXPERIMENT_NAME = "deep_cmb_colab_run"
SRC = f"experiments/{EXPERIMENT_NAME}"
DST = f"/content/drive/MyDrive/PlanckDeep/experiments/{EXPERIMENT_NAME}"

!mkdir -p "{DST}"
!cp -r {SRC}/* "{DST}/"

print("✓ Results copied to Drive:")
print(DST)


✓ Results copied to Drive:
/content/drive/MyDrive/PlanckDeep/experiments/deep_cmb_colab_run


In [7]:
# Cell X: Auto-resume from last checkpoint if available

import os

CHECKPOINT_PATH = f"/content/PlanckDeep/experiments/deep_cmb_colab_run/checkpoints"
RESUME_FLAG = ""

if os.path.exists(CHECKPOINT_PATH):
    checkpoints = [f for f in os.listdir(CHECKPOINT_PATH) if f.startswith("checkpoint_epoch_")]
    if checkpoints:
        last_checkpoint = sorted(checkpoints, key=lambda x: int(x.split("_")[-1].split(".")[0]))[-1]
        RESUME_FLAG = f"--resume {os.path.join(CHECKPOINT_PATH, last_checkpoint)}"
        print(f"✓ Resuming from checkpoint: {last_checkpoint}")
    else:
        print("⚠️ No checkpoints found. Starting fresh.")
else:
    print("🆕 No checkpoint directory found. Starting from scratch.")


🆕 No checkpoint directory found. Starting from scratch.


In [9]:
!git pull https://github.com/KhamessiTaha/PlanckDeep

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 3), reused 5 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 517 bytes | 258.00 KiB/s, done.
From https://github.com/KhamessiTaha/PlanckDeep
 * branch            HEAD       -> FETCH_HEAD
Updating 2cbf871..41ba430
Fast-forward
 src/models.py           | 2 +-
 src/train_all_models.py | 1 -
 2 files changed, 1 insertion(+), 2 deletions(-)


In [ ]:
!python train_all_models.py \
  --patch_file ../data/cmb_patches_enhanced.npy \
  --label_file ../data/cmb_temperature_labels.npy \
  --models improved_cnn resnet physics_informed \
  --epochs 20 \
  --batch_size 32 \
  --use_class_weights \
  --use_early_stopping \
  --lr_scheduler cosine \
  --experiment_name deep_cmb_colab_run \
  $RESUME_FLAG
